In [50]:
#Ejercicio 1 Importar paquetes 

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import missingno
import re
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, Normalizer
import helpers as hlp
from sklearn.decomposition import PCA

from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.tree import DecisionTreeClassifier
from sklearn.cluster import KMeans
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import classification_report
from sklearn.svm import SVC

In [51]:
df= pd.read_csv('OnlineNewsPopularity.csv')
df.head()

url   timedelta  \
0  http://mashable.com/2013/01/07/amazon-instant-...       731.0   
1  http://mashable.com/2013/01/07/ap-samsung-spon...       731.0   
2  http://mashable.com/2013/01/07/apple-40-billio...       731.0   
3  http://mashable.com/2013/01/07/astronaut-notre...       731.0   
4   http://mashable.com/2013/01/07/att-u-verse-apps/       731.0   

    n_tokens_title   n_tokens_content   n_unique_tokens   n_non_stop_words  \
0             12.0              219.0          0.663594                1.0   
1              9.0              255.0          0.604743                1.0   
2              9.0              211.0          0.575130                1.0   
3              9.0              531.0          0.503788                1.0   
4             13.0             1072.0          0.415646                1.0   

    n_non_stop_unique_tokens   num_hrefs   num_self_hrefs   num_imgs  ...  \
0                   0.815385         4.0              2.0        1.0  ...   
1                   0.791946         3.0              1.0        1.0  ...   
2                   0.663866         3.0              1.0        1.0  ...   
3                   0.665635         9.0              0.0        1.0  ...   
4                   0.540890        19.0             19.0       20.0  ...   

    min_positive_polarity   max_positive_polarity   avg_negative_polarity  \
0                0.100000                     0.7               -0.350000   
1                0.033333                     0.7               -0.118750   
2                0.100000                     1.0               -0.466667   
3                0.136364                     0.8               -0.369697   
4                0.033333                     1.0               -0.220192   

    min_negative_polarity   max_negative_polarity   title_subjectivity  \
0                  -0.600               -0.200000             0.500000   
1                  -0.125               -0.100000             0.000000   
2                  -0.800               -0.133333             0.000000   
3                  -0.600               -0.166667             0.000000   
4                  -0.500               -0.050000             0.454545   

    title_sentiment_polarity   abs_title_subjectivity  \
0                  -0.187500                 0.000000   
1                   0.000000                 0.500000   
2                   0.000000                 0.500000   
3                   0.000000                 0.500000   
4                   0.136364                 0.045455   

    abs_title_sentiment_polarity   shares  
0                       0.187500      593  
1                       0.000000      711  
2                       0.000000     1500  
3                       0.000000     1200  
4                       0.136364      505  

[5 rows x 61 columns]

In [52]:
df.shape

(39644, 61)

In [53]:
df.columns = [i.replace(' ', '') for i in df.columns]
# eliminamos el string de url que no sirve para el análisis
df = df.loc[:, 'n_tokens_title':'shares']
# generamos el conjunto de variables
qnty = df.filter(regex='^n_', axis=1)
channel = df.filter(regex='^data_', axis=1)
days = df.filter(regex=re.compile("weekday|weekend"), axis=1)
sentiments =df.filter(regex=re.compile("negative|positive|subjectivity"), axis=1)
lda = df.filter(regex='^LDA_\d', axis=1)
keywords = df.filter(regex='^kw_', axis=1)
 



In [54]:
#Analisis descriptivo

In [55]:
df

n_tokens_title  n_tokens_content  n_unique_tokens  n_non_stop_words  \
0                12.0             219.0         0.663594               1.0   
1                 9.0             255.0         0.604743               1.0   
2                 9.0             211.0         0.575130               1.0   
3                 9.0             531.0         0.503788               1.0   
4                13.0            1072.0         0.415646               1.0   
...               ...               ...              ...               ...   
39639            11.0             346.0         0.529052               1.0   
39640            12.0             328.0         0.696296               1.0   
39641            10.0             442.0         0.516355               1.0   
39642             6.0             682.0         0.539493               1.0   
39643            10.0             157.0         0.701987               1.0   

       n_non_stop_unique_tokens  num_hrefs  num_self_hrefs  num_imgs  \
0                      0.815385        4.0             2.0       1.0   
1                      0.791946        3.0             1.0       1.0   
2                      0.663866        3.0             1.0       1.0   
3                      0.665635        9.0             0.0       1.0   
4                      0.540890       19.0            19.0      20.0   
...                         ...        ...             ...       ...   
39639                  0.684783        9.0             7.0       1.0   
39640                  0.885057        9.0             7.0       3.0   
39641                  0.644128       24.0             1.0      12.0   
39642                  0.692661       10.0             1.0       1.0   
39643                  0.846154        1.0             1.0       0.0   

       num_videos  average_token_length  ...  min_positive_polarity  \
0             0.0              4.680365  ...               0.100000   
1             0.0              4.913725  ...               0.033333   
2             0.0              4.393365  ...               0.100000   
3             0.0              4.404896  ...               0.136364   
4             0.0              4.682836  ...               0.033333   
...           ...                   ...  ...                    ...   
39639         1.0              4.523121  ...               0.100000   
39640        48.0              4.405488  ...               0.136364   
39641         1.0              5.076923  ...               0.136364   
39642         0.0              4.975073  ...               0.062500   
39643         2.0              4.471338  ...               0.100000   

       max_positive_polarity  avg_negative_polarity  min_negative_polarity  \
0                       0.70              -0.350000                 -0.600   
1                       0.70              -0.118750                 -0.125   
2                       1.00              -0.466667                 -0.800   
3                       0.80              -0.369697                 -0.600   
4                       1.00              -0.220192                 -0.500   
...                      ...                    ...                    ...   
39639                   0.75              -0.260000                 -0.500   
39640                   0.70              -0.211111                 -0.400   
39641                   0.50              -0.356439                 -0.800   
39642                   0.50              -0.205246                 -0.500   
39643                   0.50              -0.200000                 -0.200   

       max_negative_polarity  title_subjectivity  title_sentiment_polarity  \
0                  -0.200000            0.500000                 -0.187500   
1                  -0.100000            0.000000                  0.000000   
2                  -0.133333            0.000000                  0.000000   
3                  -0.166667            0.000000                  0.000000   
4                  -0.050

In [56]:
hlp.identify_high_correlations(df, threshold=0.8)

variable                        var2     value
179               n_non_stop_words             n_unique_tokens  0.999572
238       n_non_stop_unique_tokens             n_unique_tokens  0.999852
239       n_non_stop_unique_tokens            n_non_stop_words  0.999532
1139                    kw_avg_min                  kw_max_min  0.940529
1256                    kw_max_max                  kw_min_min -0.857226
1499                    kw_avg_avg                  kw_max_avg  0.811864
1678    self_reference_avg_sharess   self_reference_min_shares  0.818907
1679    self_reference_avg_sharess   self_reference_max_shares  0.853480
2159                    is_weekend           weekday_is_sunday  0.701648
2196                        LDA_00         data_channel_is_bus  0.774651
2317                        LDA_02       data_channel_is_world  0.836618
2434                        LDA_04        data_channel_is_tech  0.749737
2757           rate_positive_words   global_sentiment_polarity  0.727827
2818           rate_negative_words  global_rate_negative_words  0.779556
2998         max_positive_polarity       avg_positive_polarity  0.703558
3119         min_negative_polarity       avg_negative_polarity  0.748896
3417  abs_title_sentiment_polarity          title_subjectivity  0.714528

In [57]:
#Recodificacion variable shares y eliminar variable de alta corr

df.columns = [i.replace(' ', '') for i in df.columns]
df = df.loc[:, 'n_tokens_title':'shares']
df['class'] = np.where(df['shares'] >1400, 1, 0)
y = df.pop('class')
vif = hlp.identify_high_correlations(df)
remove_vars = list(vif['variable'].values)
remove_vars.append('shares')

#Analisis de componentes principales
X_mat = df.drop(columns=remove_vars)
X_mat = PCA(n_components=30).fit_transform(X_mat)
X_train, X_test, y_train, y_test = train_test_split(X_mat, y, test_size=.33,random_state=11238)

In [59]:
#Ejercicio 2: Evaluacion de modelos individuales

m_log=LogisticRegression().fit(X_train,y_train)
m_kmedias=KMeans().fit(X_train,y_train)
m_dt1=DecisionTreeClassifier(max_depth=1).fit(X_train,y_train)
m_dt4=DecisionTreeClassifier(max_depth=4).fit(X_train,y_train)

y_pred_mlog=m_log.predict(X_test)
y_pred_kmed=m_kmedias.predict(X_test)
y_pred_dt1=m_dt1.predict(X_test)
y_pred_dt4=m_dt4.predict(X_test)
plt.subplot(2,2,1)
hlp.plot_classification_report(y_test, y_pred_mlog)
plt.axvline(.5)
plt.legend()
plt.xlim((0.1, 1))


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


ValueError: cannot reshape array of size 23 into shape (4,5)

In [ ]:
estimators = [
('logisticl1', LogisticRegression()),
('Linear Discriminant Analysis',LinearDiscriminantAnalysis()),
('decstump', DecisionTreeClassifier(max_depth=1)),
('dectree4', DecisionTreeClassifier(max_depth=4))
]

In [ ]:
#Analisis descriptivo de atributos

In [ ]:
#Correlacion de atributos


data_corr = hlp.identify_high_correlations(df)
variables_eliminar = data_corr[np.abs(data_corr['value'])>=0.8]['var2'].values
df.drop(variables_eliminar,axis=1)

n_tokens_title  n_tokens_content  n_non_stop_unique_tokens  num_hrefs  \
0                12.0             219.0                  0.815385        4.0   
1                 9.0             255.0                  0.791946        3.0   
2                 9.0             211.0                  0.663866        3.0   
3                 9.0             531.0                  0.665635        9.0   
4                13.0            1072.0                  0.540890       19.0   
...               ...               ...                       ...        ...   
39639            11.0             346.0                  0.684783        9.0   
39640            12.0             328.0                  0.885057        9.0   
39641            10.0             442.0                  0.644128       24.0   
39642             6.0             682.0                  0.692661       10.0   
39643            10.0             157.0                  0.846154        1.0   

       num_self_hrefs  num_imgs  num_videos  average_token_length  \
0                 2.0       1.0         0.0              4.680365   
1                 1.0       1.0         0.0              4.913725   
2                 1.0       1.0         0.0              4.393365   
3                 0.0       1.0         0.0              4.404896   
4                19.0      20.0         0.0              4.682836   
...               ...       ...         ...                   ...   
39639             7.0       1.0         1.0              4.523121   
39640             7.0       3.0        48.0              4.405488   
39641             1.0      12.0         1.0              5.076923   
39642             1.0       1.0         0.0              4.975073   
39643             1.0       0.0         2.0              4.471338   

       num_keywords  data_channel_is_lifestyle  ...  min_positive_polarity  \
0               5.0                        0.0  ...               0.100000   
1               4.0                        0.0  ...               0.033333   
2               6.0                        0.0  ...               0.100000   
3               7.0                        0.0  ...               0.136364   
4               7.0                        0.0  ...               0.033333   
...             ...                        ...  ...                    ...   
39639           8.0                        0.0  ...               0.100000   
39640           7.0                        0.0  ...               0.136364   
39641           8.0                        0.0  ...               0.136364   
39642           5.0                        0.0  ...               0.062500   
39643           4.0                        0.0  ...               0.100000   

       max_positive_polarity  avg_negative_polarity  min_negative_polarity  \
0                       0.70              -0.350000                 -0.600   
1                       0.70              -0.118750                 -0.125   
2                       1.00              -0.466667                 -0.800   
3                       0.80              -0.369697                 -0.600   
4                       1.00              -0.220192                 -0.500   
...                      ...                    ...                    ...   
39639                   0.75              -0.260000                 -0.500   
39640                   0.70              -0.211111                 -0.400   
39641                   0.50              -0.356439                 -0.800   
39642                   0.50              -0.205246                 -0.500   
39643                   0.50              -0.200000                 -0.200   

       max_negative_polarity  title_subjectivity  title_sentiment_polarity  \
0                  -0.200000            0.500000                 -0.187500   
1                  -0.100000            0.000000                  0.000000   
2                  -0.133333            0.000000                  0.000000   
3                  -0.166667   

In [ ]:
#Segun las metricas del modelos ponderarlos mas o menos

In [ ]:
#Ver dedesmpe;no individial y acorde a eso proponer ponderadores

voting_model=VotingClassifier(estimators,weights=[0.3,0.3,0.3,0.1]).fit(X_train,y_train)